<a href="https://colab.research.google.com/github/mltrev23/tech-test/blob/main/7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem
7. Deployment using Hugging Face Transformers, 4 pipelines for tasks related to:
 - Sentiment analysis.
 - Text summarization.
 - Document classification.
 - Automatic translation.

Setup environment

In [ ]:
pip install transformers torch fastapi uvicorn pyngrok nest_asyncio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 4.4 MB/s eta 0:00:00


Implement the Pipelines

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
from transformers import pipeline

# Initialize FastAPI app
app = FastAPI()

Define the Pipelines

In [ ]:
# Load the pipelines
sentiment_analysis_pipeline = pipeline('sentiment-analysis')
summarization_pipeline = pipeline("summarization")
document_classification_pipeline = pipeline('text-classification')
translation_pipeline = pipeline('translation_en_to_fr')

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Define Input Data Models

In [ ]:
class SentimentAnalysisData(BaseModel):
    text: str

class SummarizationData(BaseModel):
    text: str

class DocumentClassificationData(BaseModel):
    text: str

class TranslationData(BaseModel):
    text: str

Define the Routes

In [ ]:
@app.post('/sentiment-analysis')
def sentiment_analysis(data: SentimentAnalysisData):
    result = sentiment_analysis_pipeline(data.text)
    print(result)
    return {"sentiment": result}

@app.post('/summarize')
def summarize(data: SummarizationData):
    result = summarization_pipeline(data.text)
    print(result[0]['summary_text'])
    return {"summary": result[0]['summary_text']}

@app.post('/document-classification')
def classify_document(data: DocumentClassificationData):
    result = document_classification_pipeline(data.text)
    print(result)
    return {"classification": result}

@app.post('/translate')
def translate(data: TranslationData):
    result = translation_pipeline(data.text)
    print(result[0]['translation_text'])
    return {"translation": result[0]['translation_text']}

Run the FastAPI App

In [ ]:
!ngrok authtoken 2lammpPuDThaunJy6qYTw0YVylF_6XJrWtbYB8dm1XMVY2qh1

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import uvicorn
import nest_asyncio
from pyngrok import ngrok

# Apply the nest_asyncio patch
nest_asyncio.apply()

public_url = ngrok.connect(9007, "http")
print('Public URL:', public_url)

uvicorn.run(app, host='0.0.0.0', port=9007)

Public URL: NgrokTunnel: "https://3566-34-168-133-231.ngrok-free.app" -> "http://localhost:9007"


INFO:     Started server process [417]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:9007 (Press CTRL+C to quit)
Your max_length is set to 142, but your input_length is only 58. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=29)


INFO:     204.44.96.131:0 - "POST /summarize HTTP/1.1" 200 OK
INFO:     204.44.96.131:0 - "POST /sentiment-analysis HTTP/1.1" 200 OK
INFO:     204.44.96.131:0 - "POST /document-classification HTTP/1.1" 200 OK
INFO:     204.44.96.131:0 - "POST /translate HTTP/1.1" 200 OK
